In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

def convert_sample(sample):
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image, image  # Use the image itself as the target label

def map_func(sample):
    image = sample['image']
    label = sample['label']
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label


# Corrected file path with escaped backslashes and a missing comma
ds1, ds2, ds3 = tfds.load('patch_camelyon',
                         split=['train[:10%]', 'test[:2.5%]', 'validation[:2.5%]'],
                         data_dir=r'C:\Job_og_eksamensbevis\Github\projekter\Projekt_1_Computer_Vision\path_to_data_directory',
                         download=False,
                         shuffle_files=True)

train_dataset       = ds1.map(convert_sample).batch(32)
validation_dataset  = ds3.map(convert_sample).batch(32)
test_dataset        = ds2.map(convert_sample).batch(32)

train_dataset_cnn       = ds1.map(map_func).batch(32)
validation_dataset_cnn  = ds3.map(map_func).batch(32)
test_dataset_cnn        = ds2.map(map_func).batch(32)


c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\runpy.py", line 86, in _run_code
    exec(code, run_

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\jeppe\anaconda3\a3\envs\portfolioJFR\lib\site-packages\traitlets\config\application.py", line 10

AttributeError: _ARRAY_API not found

ValueError: Error parsing split 'test[:2.5%]'. See format at: https://www.tensorflow.org/datasets/splits
Unrecognized split format: 'test[:2.5%]'. See format at https://www.tensorflow.org/datasets/splits

In [ ]:

#baseline

from tensorflow.keras.callbacks import TensorBoard, 
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D, Conv2DTranspose 

def build_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded_layer = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded_layer)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded_layer = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoded_layer)

    return autoencoder

# Assuming input image shape is (96, 96, 3) for a 96x96px color image
input_shape = (96, 96, 3)
autoencoder = build_autoencoder(input_shape)


# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')  # You can use other loss functions depending on your task


# Train the autoencoder
history = autoencoder.fit(train_dataset,
                          validation_data=validation_dataset,
                          epochs=10,
                          batch_size=32
                          )


# Evaluate performance (you may use a validation set or a separate test set)
loss = autoencoder.evaluate(test_dataset, verbose=1)


# Access the training history
loss = history.history['loss']
val_loss = history.history['val_loss']


# Plot the training progress
import matplotlib.pyplot as plt

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [3]:
#preferred AE-model
#AE med dropout

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, UpSampling2D, Dropout, Conv2DTranspose 

def build_autoencoder(input_shape, dropout_rate=0.2):
    input_layer = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    encoded_layer = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded_layer)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Dropout(dropout_rate)(x)  # Dropout layer added here
    decoded_layer = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoded_layer)

    return autoencoder

# Assuming input image shape is (96, 96, 3) for a 96x96px color image
input_shape = (96, 96, 3)
autoencoder = build_autoencoder(input_shape)
dropout_rate = 0.2  # Modify the dropout rate as per your preference 
early_stopping = EarlyStopping(monitor='val_loss', patience=3) 

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')  

# Train the autoencoder
history = autoencoder.fit(train_dataset,
                          validation_data=validation_dataset,
                          epochs=10,
                          batch_size=32
                          )

# Evaluate performance (you may use a validation set or a separate test set)
loss = autoencoder.evaluate(test_dataset, verbose=1)


# Access the training history
loss = history.history['loss']
val_loss = history.history['val_loss']


# Plot the training progress
import matplotlib.pyplot as plt

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/10



820/820 [==============================] - 119s 143ms/step - loss: 0.0195 - val_loss: 0.0146
Epoch 2/10
820/820 [==============================] - 114s 138ms/step - loss: 0.0132 - val_loss: 0.0125
Epoch 3/10
789/820 [===========================>..] - ETA: 4s - loss: 0.0120

KeyboardInterrupt: 

In [ ]:
#reconstruct images

import matplotlib.pyplot as plt
import tensorflow as tf

# Antal billeder at vise
n = 10

# Hent nogle eksempler fra testdatasættet
samples = next(iter(test_dataset))[0][:n]  # Brug kun inputbilleder

# Generer rekonstruerede billeder fra de originale billeder
reconstructed_images = autoencoder.predict(samples)

plt.figure(figsize=(20, 4))
for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.title("Original")
    plt.imshow(tf.squeeze(samples[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display rekonstruktion
    bx = plt.subplot(2, n, i + 1 + n)
    plt.title("Rekonstrueret")
    plt.imshow(tf.squeeze(reconstructed_images[i]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)

plt.show()

from sklearn.metrics import mean_squared_error
import tensorflow as tf

# Assuming 'samples' contains original images and 'reconstructed_images' contains corresponding reconstructed images

# Reshape the arrays to 2D (flatten each image)
samples_flat = tf.reshape(samples, (samples.shape[0], -1))
reconstructed_flat = tf.reshape(reconstructed_images, (reconstructed_images.shape[0], -1))

# Convert EagerTensor to numpy arrays
samples_flat = samples_flat.numpy()
reconstructed_flat = reconstructed_flat.numpy()

# Calculate Mean Squared Error
mse = mean_squared_error(samples_flat, reconstructed_flat)
print(f"Mean Squared Error: {mse}")

